In [2]:
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
pd.options.display.max_rows = 100
pd.options.display.max_columns = 40
import numpy as np
import os,random, math
from tqdm import tqdm

import datetime

from dateutil.relativedelta import relativedelta
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error as mae
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, Ridge

import os
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
from fbprophet import Prophet
from fbprophet.diagnostics import performance_metrics
from fbprophet.diagnostics import cross_validation

data = pd.read_csv('/Users/seungji/Desktop/JejuECO/x_predict/0911최종/final_merged_by_day_v6.csv')

data["week"] = pd.DatetimeIndex(data.base_date).to_period("W")
data["week_num"] = 0
for idx,w in enumerate (data.week.unique()):
    data.loc[data.week == w,"week_num"] = idx
data = data.drop(columns = 'week')

holiday = data.groupby(['base_date']).mean()['holidays']
data.base_date = pd.to_datetime(data.base_date)
data.index = data.base_date
df = data

col = ['korean_resd_pop', 'foreign_resd_pop', 'total_resd_pop','foreign_resd_ratio',
       'fam', 'pop_per_fam', 'popdens',
        '기온', '습도', '풍속', '태풍', '미세먼지',
        '검색_관광', '검색_쇼핑', '검색_쓰레기', '검색_한파', '검색_강풍', '검색_태풍',
       '검색_장마', '검색_폭염', '검색_우박', '검색_날씨', '검색_미세먼지', '검색_황사', '읍면동별검색_생활',
       '읍면동별검색_관광', '읍면동별검색_쇼핑', '총계_월', '내국인전체_월', '외국인_월', '연령0020',
       '연령2030', '연령2040_rt', '연령3040', '연령4060', '연령60이상', 'week_num', 'holidays'
      ]
df['vpc_2050_daytm'] = df.vpc_daytm_rt*df.visit_pop_jeju

In [3]:
def merge (train,test):
    data = pd.concat([train,test],axis = 0)
    data["week"] = pd.DatetimeIndex(data.base_date).to_period("W")
    data["week_num"] = 0
    for idx,w in enumerate (data.week.unique()):
        data.loc[data.week == w,"week_num"] = idx
    data = data.drop(columns = 'week')
    data.base_date = pd.to_datetime(data.base_date)
    data.index = data.base_date
    return data

In [4]:
train = pd.read_csv('/Users/seungji/Desktop/JejuECO/x_predict/0911최종/final_merged_by_day_v6.csv')
train.base_date = pd.to_datetime(train.base_date)
test = pd.read_csv('/Users/seungji/Desktop/JejuECO/final_merged_by_day_v6_7_8.csv')
test.base_date = pd.to_datetime(test.base_date)
df = merge(train,test)

# test["week"] = pd.DatetimeIndex(test.base_date).to_period("W")
# test["week_num"] = 0
# for idx,w in enumerate (test.week.unique()):
#     test.loc[test.week == w,"week_num"] = idx
# test = test.drop(columns = 'week')
# test.base_date = pd.to_datetime(test.base_date)
# test.index = test.base_date

df['vpc_2050_daytm'] = df.visit_pop_jeju * df.vpc_daytm_rt

In [5]:
df

,base_date,emd_nm,pay_amt,em_g,em_cnt,city,cluster,holidays,korean_resd_pop,foreign_resd_pop,total_resd_pop,foreign_resd_ratio,fam,pop_per_fam,popdens,resd_pop_cnt,work_pop_jeju,work_pop_etc,visit_pop_jeju,visit_pop_etc,...,검색_폭염,검색_우박,검색_날씨,검색_미세먼지,검색_황사,읍면동별검색_생활,읍면동별검색_관광,읍면동별검색_쇼핑,총계_월,내국인전체_월,외국인_월,연령0020,연령2030,연령2040_rt,연령3040,연령4060,연령60이상,vpc_daytm_rt,week_num,vpc_2050_daytm
base_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-01-01,2018-01-01,남원읍,"37,110.0000","1,239,600.0000",304.0000,서귀포시,seo_3,1,"19,112.0000",628.0000,"19,740.0000",0.0318,8345,2.3655,104.3947,"2,963.1062",461.6338,86.4137,"4,160.2772","2,235.6352",...,2.5395,0.0000,4.7375,0.6964,0.4327,2.9925,63.8883,18.6833,"1,102,378.0000","1,044,956.0000","57,422.0000","3,075.0000","2,116.0000",0.2086,"1,871.0000","6,496.0000","5,554.0000",0.6912,0,"2,875.3773"
2018-01-01,2018-01-01,대륜동,"50,076.0000","1,717,700.0000",650.0000,서귀포시,seo_3,1,"13,473.0000",178.0000,"13,651.0000",0.0130,5586,2.4438,613.8040,"2,757.4585",437.8499,68.0000,"3,833.0170","1,963.9697",...,2.5395,0.0000,4.7375,0.6964,0.4327,10.1852,11.6279,0.0000,"1,102,378.0000","1,044,956.0000","57,422.0000","2,732.0000","1,422.0000",0.2563,"2,031.0000","4,363.0000","2,925.0000",0.5851,0,"2,242.8398"
2018-01-01,2018-01-01,대정읍,"37,872.0000","1,264,950.0000",309.0000,서귀포시,seo_1,1,"21,085.0000","1,518.0000","22,603.0000",0.0672,9701,2.3300,287.1316,"3,782.3534",281.9346,91.2229,"3,553.5190","1,328.5841",...,2.5395,0.0000,4.7375,0.6964,0.4327,15.2778,63.5907,0.0000,"1,102,378.0000","1,044,956.0000","57,422.0000","4,294.0000","1,947.0000",0.1997,"2,264.0000","7,250.0000","5,330.0000",0.6493,0,"2,307.4152"
2018-01-01,2018-01-01,대천동,"52,008.0000","1,765,600.0000",696.0000,서귀포시,seo_3,1,"13,160.0000",244.0000,"13,404.0000",0.0182,5770,2.3231,263.3916,"1,580.4708",172.9991,33.3392,"2,816.0447","1,274.2927",...,2.5395,0.0000,4.7375,0.6964,0.4327,15.2174,40.8502,16.6352,"1,102,378.0000","1,044,956.0000","57,422.0000","2,839.0000","1,456.0000",0.2891,"2,349.0000","4,144.0000","2,372.0000",0.5749,0,"1,619.0433"
2018-01-01,2018-01-01,동홍동,"122,434.0000","4,165,050.0000","1,567.0000",서귀포시,seo_3,1,"22,693.0000","1,266.0000","23,959.0000",0.0528,8648,2.7705,"1,675.4545","3,813.0625",458.3857,48.2290,"3,813.4420",972.5606,...,2.5395,0.0000,4.7375,0.6964,0.4327,0.0000,12.9555,0.0000,"1,102,378.0000","1,044,956.0000","57,422.0000","5,702.0000","2,707.0000",0.2392,"2,722.0000","7,664.0000","3,898.0000",0.5670,0,"2,162.3509"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-31,2021-08-31,표선면,NaN,NaN,NaN,NaN,NaN,0,"11,999.0000",341.0000,"12,340.0000",0.0276,6175,2.0000,91.2857,NaN,NaN,NaN,NaN,NaN,...,4.0249,0.0000,32.0493,0.0867,0.1082,31.0811,83.0486,42.2535,"986,408.0000","982,032.0000","4,376.0000","1,782.0000","1,197.0000",0.1840,"1,073.0000","4,301.0000","3,987.0000",NaN,191,NaN
2021-08-31,2021-08-31,한경면,NaN,NaN,NaN,NaN,NaN,0,"9,229.0000",346.0000,"9,575.0000",0.0361,4938,1.8700,121.0187,NaN,NaN,NaN,NaN,NaN,...,4.0249,0.0000,32.0493,0.0867,0.1082,16.6667,113.4645,0.0000,"986,408.0000","982,032.0000","4,376.0000","1,115.0000",746.0000,0.1691,815.0000,"3,093.0000","3,460.0000",NaN,191,NaN
2021-08-31,2021-08-31,한림읍,NaN,NaN,NaN,NaN,NaN,0,"21,203.0000","3,143.0000","24,346.0000",0.1291,10546,2.0100,266.9225,NaN,NaN,NaN,NaN,NaN,...,4.0249,0.0000,32.0493,0.0867,0.1082,30.4617,123.1717,9.3923,"986,408.0000","982,032.0000","4,376.0000","3,264.0000","2,035.0000",0.1939,"2,076.0000","7,321.0000","6,507.0000",NaN,191,NaN


In [6]:
data = pd.concat([train,test],axis = 0)
data.to_csv('/Users/seungji/Desktop/JejuECO/x_predict/0911최종/data.csv',index = False)
df = data

In [7]:
data.columns

Index(['base_date', 'emd_nm', 'pay_amt', 'em_g', 'em_cnt', 'city', 'cluster',
       'holidays', 'korean_resd_pop', 'foreign_resd_pop', 'total_resd_pop',
       'foreign_resd_ratio', 'fam', 'pop_per_fam', 'popdens', 'resd_pop_cnt',
       'work_pop_jeju', 'work_pop_etc', 'visit_pop_jeju', 'visit_pop_etc',
       'visit_short', 'CHN_short', 'JPN_short', 'USA_short', 'CHN_ratio_short',
       'NotCHN_short', 'resd_jeju', 'country_1_all_jeju',
       'country_2_resd_jeju', 'country_2_visit_jeju', 'country_3_resd_jeju',
       'country_4_resd_jeju', 'country_7_all_jeju', 'country_3_all_etc', '강수',
       '기온', '습도', '풍속', '태풍', '미세먼지', '유형1_cnt', '유형1_amt', '유형2_cnt',
       '유형2_amt', '유형3_cnt', '유형3_amt', '농축수산물_cnt', '농축수산물_amt', '배달_cnt',
       '배달_amt', '배달_ratio', '유형1_ratio', '유형2_ratio', '유형3_ratio',
       '농축수산물_ratio', '검색_관광', '검색_쇼핑', '검색_쓰레기', '검색_한파', '검색_강풍', '검색_태풍',
       '검색_장마', '검색_폭염', '검색_우박', '검색_날씨', '검색_미세먼지', '검색_황사', '읍면동별검색_생활',
       '읍면동별검색_관광', '읍면동별검색_쇼핑

In [8]:
def split_ts (df, c1, c2 = "2021-09-01",start0 = "2018-01-01"):
    tr = df[df.base_date <c1]
    tr = tr[tr.base_date>=start0]
    tst = df[df.base_date >=c1]
    tst = tst[tst.base_date<c2]
    return tr, tst

def MAPE(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100 

In [9]:
base_col = ['base_date', 'emd_nm', 'pay_amt', 'em_g', 'em_cnt','city','cluster']

time_col = ['holidays']

pop_col = ['korean_resd_pop', 'foreign_resd_pop', 'total_resd_pop','foreign_resd_ratio',
           'fam', 'pop_per_fam', 'popdens']
        
pop_col_korean = ['resd_pop_cnt', 'work_pop_jeju', 'work_pop_etc', 'visit_pop_jeju','visit_pop_etc','vpc_daytm_rt']

pop_col_foreign = ['visit_short','CHN_short', 'JPN_short', 'USA_short', 'CHN_ratio_short','NotCHN_short', 
                   'resd_jeju','country_1_all_jeju','country_2_resd_jeju', 'country_2_visit_jeju', 'country_3_resd_jeju',
                   'country_4_resd_jeju', 'country_7_all_jeju', 'country_3_all_etc']

weather_col = ['강수','기온','습도','풍속','태풍','미세먼지']

card_col = ['유형1_cnt', '유형1_amt',  '유형2_cnt', '유형2_amt',  '유형3_cnt',
            '유형3_amt', '농축수산물_cnt', '농축수산물_amt','배달_cnt', '배달_amt', 
            '배달_ratio','유형1_ratio','유형2_ratio','유형3_ratio', '농축수산물_ratio']

search_col = ['검색_관광', '검색_쇼핑', '검색_쓰레기', '검색_한파', '검색_강풍', '검색_태풍', '검색_장마', '검색_폭염',
               '검색_우박', '검색_날씨', '검색_미세먼지', '검색_황사', '읍면동별검색_생활', '읍면동별검색_관광',
               '읍면동별검색_쇼핑']

entrance_col = ['총계_월', '내국인전체_월', '외국인_월']

age = ['연령0020', '연령2030', '연령2040_rt','연령3040', '연령4060', '연령60이상']
col = time_col+pop_col+weather_col+search_col+entrance_col+age

In [10]:
holiday = df.groupby([df.index]).mean()['holidays']
holidays =holiday.reset_index()
holidays.columns = ['ds','holiday']

national = pd.DataFrame({
    'holiday':'national',
    'ds':pd.to_datetime(holidays.ds),
    'lower_window': 0,'upper_window': 1
})

holidays = national

In [11]:
x_model = pd.read_csv('/Users/seungji/Desktop/JejuECO/x_predict/0911최종/x_model.csv')
x_model = x_model.replace({'nan':'np.nan'})

In [12]:
def modeling(emd, x_var,col, var,tm,mod, start0 = '2018-01-01', end = '2021-09-01', scale=False):
    if tm == 'corona': start0 = '2019-07-01'
    tmp = df.loc[df.emd_nm == emd,["base_date",x_var]]
    tmp.index = tmp.base_date
    train = tmp[:"2021-07-01"]
    train.columns = ["ds", "y"]
    
    model = Prophet(weekly_seasonality=True, daily_seasonality=True, yearly_seasonality=True, 
                            changepoint_prior_scale=0.05, seasonality_mode='multiplicative',holidays = holidays)
    model.fit(train)
    
    
    future = model.make_future_dataframe(periods=63)
    forecast = model.predict(future)
    forecast.index = forecast.ds
    
    tmp = df[(df['emd_nm']== emd)&(df.base_date>=start0)&(df.base_date<=end)]
    tmp.index = pd.to_datetime(tmp.base_date)
    #print(tmp.base_date.unique());print('**************')
    #print(forecast.loc[forecast.index.isin(tmp.base_date),'ds'])
    tmp["trend"] = forecast.loc[forecast.index.isin(tmp.base_date),'trend'].values
    tmp["yhat"] = forecast.loc[forecast.index.isin(tmp.base_date),'yhat'].values
    tmp["weekly"] = forecast.loc[forecast.index.isin(tmp.base_date),'weekly'].values

    if var == "yhat":
        x2 = col + ["yhat"]
    else:
        if var == "trend_wk":
            x2 = col + ["trend","weekly"]
    
    train, test = split_ts(tmp, c1 = "2021-07-01", c2 = end, start0 = start0)
    bd_tst = test.base_date.unique()
    X_train = train[x2]
    X_test = test[x2]
    
    if scale:
        scalar = StandardScaler()
        scalar.fit(X_train)
        X_train = scalar.transform(X_train)
        X_test = scalar.transform(X_test)

    y_train = train[[x_var]]
    y_test = test[[x_var]]
    
    if scale: y_train[[x_var]] = y_train[[x_var]].apply(lambda x : np.log(x))
    
    if mod == "rdg":
        model = Ridge()
    else:
        if mod == "rf":
            model = RandomForestRegressor()
    #print(X_train); print(y_train)
    lr= model.fit(X_train, y_train)
    #print(X_test)
    pred = lr.predict(X_test)

    res = pd.DataFrame(pred,columns=['pred'])
    if scale: res['pred'] = res['pred'].apply(lambda x : np.exp(x))
    #res[x_var] = y_test[x_var].to_list()
    res["base_date"] = bd_tst
    return res

In [13]:
x_var = 'resd_pop_cnt'
way = dict(zip(x_model[-x_model[x_var].isna()].emd_nm, x_model[-x_model[x_var].isna()][x_var]))
output = pd.DataFrame()
for emd in way.keys():
    print(emd)
    var = way[emd].split("'")[1]
    tm = way[emd].split("'")[3].split('_')[1]
    mod = way[emd].split("'")[3].split('_')[0]
    res = modeling(emd,x_var,col,var,tm,mod)
    res['emd_nm'] = emd
    res = res.rename(columns = {'pred':x_var})
    output = pd.concat([output,res],axis=0)
test = pd.merge(test, output,on = ['base_date','emd_nm'],how = 'left')

건입동
구좌읍
남원읍
노형동
대륜동
대정읍
대천동
도두동
동홍동
봉개동
삼도1동
삼도2동
삼양동
서홍동
성산읍
송산동
아라동
안덕면
애월읍
연동
영천동
예래동
오라동
외도동
용담1동
용담2동
이도1동
이도2동
이호동
일도1동
일도2동
정방동
조천읍
중문동
중앙동
천지동
표선면
한경면
한림읍
화북동
효돈동


test = test.reset_index(drop=True)
test = pd.merge(test,result, on = ['base_date','emd_nm'], how = 'left')

In [14]:
df = merge(train,test)

###### visit_pop_jeju, work_pop_jeju 예측

In [15]:
col += ['resd_pop_cnt']
#result = pd.DataFrame()
for x_var in ['visit_pop_jeju','work_pop_jeju']:
    way = dict(zip(x_model[-x_model[x_var].isna()].emd_nm, x_model[-x_model[x_var].isna()][x_var]))
    output = pd.DataFrame()
    for emd in way.keys():
        print(emd)
        var = way[emd].split("'")[1]
        tm = way[emd].split("'")[3].split('_')[1]
        mod = way[emd].split("'")[3].split('_')[0]
        res = modeling(emd,x_var,col,var,tm,mod)
        res['emd_nm'] = emd
        res = res.rename(columns = {'pred':x_var})
        output = pd.concat([output,res],axis=0)
    test = pd.merge(test, output,on = ['base_date','emd_nm'],how = 'left')

건입동
구좌읍
남원읍
노형동
대륜동
대정읍
대천동
도두동
동홍동
봉개동
삼도1동
삼도2동
삼양동
서홍동
성산읍
송산동
아라동
안덕면
애월읍
연동
영천동
예래동
오라동
외도동
용담1동
용담2동
이도1동
이도2동
이호동
일도1동
일도2동
정방동
조천읍
중문동
중앙동
천지동
표선면
한경면
한림읍
화북동
효돈동
건입동
구좌읍
남원읍
노형동
대륜동
대정읍
대천동
도두동
동홍동
봉개동
삼도1동
삼도2동
삼양동
서홍동
성산읍
송산동
아라동
안덕면
애월읍
연동
영천동
예래동
오라동
외도동
용담1동
용담2동
이도1동
이도2동
이호동
일도1동
일도2동
정방동
조천읍
중문동
중앙동
천지동
표선면
한경면
한림읍
화북동
효돈동


test = test.reset_index(drop=True)
test = pd.merge(test,result, on = ['base_date','emd_nm'], how = 'left')

In [16]:
df = merge(train,test)

###### visit_pop_etc 예측

In [19]:
col += ['visit_pop_jeju','work_pop_jeju']
for x_var in ['visit_pop_etc']:
    way = dict(zip(x_model[-x_model[x_var].isna()].emd_nm, x_model[-x_model[x_var].isna()][x_var]))
    output = pd.DataFrame()
    for emd in way.keys():
        print(emd)
        var = way[emd].split("'")[1]
        tm = way[emd].split("'")[3].split('_')[1]
        mod = way[emd].split("'")[3].split('_')[0]
        res = modeling(emd,x_var,col,var,tm,mod)
        res['emd_nm'] = emd
        res = res.rename(columns = {'pred':x_var})
        output = pd.concat([output,res],axis=0)
    test = pd.merge(test, output,on = ['base_date','emd_nm'],how = 'left')

건입동
구좌읍
남원읍
노형동
대정읍
대천동
동홍동
봉개동
삼도1동
삼도2동
삼양동
서홍동
성산읍
송산동
아라동
안덕면
애월읍
연동
영천동
오라동
외도동
용담1동
이도1동
이도2동
일도2동
조천읍
중문동
중앙동
표선면
한경면
한림읍


test = test.reset_index(drop=True)
test = pd.merge(test,result, on = ['base_date','emd_nm'], how = 'left')

In [20]:
df = merge(train,test)

###### work_pop_etc 예측

In [22]:
df.isnull().sum()

base_date                  0
emd_nm                     0
pay_amt                 2542
em_g                    2542
em_cnt                  2542
city                    2542
cluster                 2542
holidays                   0
korean_resd_pop            0
foreign_resd_pop           0
total_resd_pop             0
foreign_resd_ratio         0
fam                        0
pop_per_fam                0
popdens                    0
resd_pop_cnt               0
work_pop_jeju              0
work_pop_etc            2542
visit_pop_jeju             0
visit_pop_etc            620
visit_short             2542
CHN_short               2542
JPN_short               2542
USA_short               2542
CHN_ratio_short         2542
NotCHN_short            2542
resd_jeju               2542
country_1_all_jeju      2542
country_2_resd_jeju     2542
country_2_visit_jeju    2542
country_3_resd_jeju     2542
country_4_resd_jeju     2542
country_7_all_jeju      2542
country_3_all_etc       2542
강수            

In [23]:
emdt = df[-df.visit_pop_etc.isnull()].emd_nm.unique()
emdt 

array(['남원읍', '대륜동', '대정읍', '대천동', '동홍동', '서홍동', '성산읍', '송산동', '안덕면',
       '영천동', '예래동', '정방동', '중문동', '중앙동', '천지동', '표선면', '효돈동', '건입동',
       '노형동', '도두동', '봉개동', '삼도1동', '삼도2동', '삼양동', '아라동', '애월읍', '연동',
       '오라동', '외도동', '용담1동', '용담2동', '이도1동', '이도2동', '이호동', '일도1동',
       '일도2동', '화북동', '조천읍', '구좌읍', '한림읍', '한경면'], dtype=object)

In [24]:
way = dict(zip(x_model[-x_model['work_pop_etc'].isna()].emd_nm, x_model[-x_model['work_pop_etc'].isna()]['work_pop_etc']))
way.keys()

dict_keys(['남원읍', '노형동', '대륜동', '대정읍', '도두동', '봉개동', '삼도2동', '서홍동', '성산읍', '송산동', '안덕면', '애월읍', '연동', '예래동', '이도1동', '이도2동', '이호동', '일도1동', '정방동', '조천읍', '중문동', '중앙동', '천지동', '표선면', '한림읍', '화북동'])

In [25]:
col1 = col + ['visit_pop_etc']
for x_var in ['work_pop_etc']:
    way = dict(zip(x_model[-x_model[x_var].isna()].emd_nm, x_model[-x_model[x_var].isna()][x_var]))
    output = pd.DataFrame()
    for emd in way.keys():
        if emd in emdt:
            print(emd)
            var = way[emd].split("'")[1]
            tm = way[emd].split("'")[3].split('_')[1]
            mod = way[emd].split("'")[3].split('_')[0]
            res = modeling(emd,x_var,col,var,tm,mod)
            res['emd_nm'] = emd
            res = res.rename(columns = {'pred':x_var})
            output = pd.concat([output,res],axis=0)
        else:
            print(emd)
            var = way[emd].split("'")[1]
            tm = way[emd].split("'")[3].split('_')[1]
            mod = way[emd].split("'")[3].split('_')[0]
            res = modeling(emd,x_var,col1,var,tm,mod)
            res['emd_nm'] = emd
            res = res.rename(columns = {'pred':x_var})
            output = pd.concat([output,res],axis=0)
    test = pd.merge(test, output,on = ['base_date','emd_nm'],how = 'left')

남원읍
노형동
대륜동
대정읍
도두동
봉개동
삼도2동
서홍동
성산읍
송산동
안덕면
애월읍
연동
예래동
이도1동
이도2동
이호동
일도1동
정방동
조천읍
중문동
중앙동
천지동
표선면
한림읍
화북동


test = test.reset_index(drop=True)
test = pd.merge(test,result, on = ['base_date','emd_nm'], how = 'left')

In [26]:
df = merge(train,test)

In [27]:
df[df.emd_nm.isin(['남원읍'])]

,base_date,emd_nm,pay_amt,em_g,em_cnt,city,cluster,holidays,korean_resd_pop,foreign_resd_pop,total_resd_pop,foreign_resd_ratio,fam,pop_per_fam,popdens,resd_pop_cnt,work_pop_jeju,work_pop_etc,visit_pop_jeju,visit_pop_etc,...,검색_장마,검색_폭염,검색_우박,검색_날씨,검색_미세먼지,검색_황사,읍면동별검색_생활,읍면동별검색_관광,읍면동별검색_쇼핑,총계_월,내국인전체_월,외국인_월,연령0020,연령2030,연령2040_rt,연령3040,연령4060,연령60이상,vpc_daytm_rt,week_num
base_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-01-01,2018-01-01,남원읍,"37,110.0000","1,239,600.0000",304.0000,서귀포시,seo_3,1,"19,112.0000",628.0000,"19,740.0000",0.0318,8345,2.3655,104.3947,"2,963.1062",461.6338,86.4137,"4,160.2772","2,235.6352",...,0.0562,2.5395,0.0000,4.7375,0.6964,0.4327,2.9925,63.8883,18.6833,"1,102,378.0000","1,044,956.0000","57,422.0000","3,075.0000","2,116.0000",0.2086,"1,871.0000","6,496.0000","5,554.0000",0.6912,0
2018-01-02,2018-01-02,남원읍,"47,002.0000","1,569,850.0000",381.0000,서귀포시,seo_3,0,"19,112.0000",628.0000,"19,740.0000",0.0318,8345,2.3655,104.3947,"2,959.4985",883.7714,85.7563,"3,492.6830","1,836.5883",...,0.0984,3.6895,0.0000,6.0363,0.7067,0.3894,7.1607,48.3513,9.0747,"1,102,378.0000","1,044,956.0000","57,422.0000","3,075.0000","2,116.0000",0.2086,"1,871.0000","6,496.0000","5,554.0000",0.7480,0
2018-01-03,2018-01-03,남원읍,"42,080.0000","1,405,650.0000",336.0000,서귀포시,seo_3,0,"19,112.0000",628.0000,"19,740.0000",0.0318,8345,2.3655,104.3947,"2,921.7790",903.2848,79.3105,"3,824.1481","1,972.6704",...,0.0996,3.7853,0.0000,7.1880,0.4393,0.1947,5.4863,49.9227,9.9644,"1,102,378.0000","1,044,956.0000","57,422.0000","3,075.0000","2,116.0000",0.2086,"1,871.0000","6,496.0000","5,554.0000",0.7650,0
2018-01-04,2018-01-04,남원읍,"41,610.0000","1,390,250.0000",355.0000,서귀포시,seo_3,0,"19,112.0000",628.0000,"19,740.0000",0.0318,8345,2.3655,104.3947,"2,831.7277",879.3167,77.6560,"3,605.5431","2,097.4130",...,0.2120,3.0187,0.0000,9.0050,0.3746,0.2813,5.1300,51.9948,7.2954,"1,102,378.0000","1,044,956.0000","57,422.0000","3,075.0000","2,116.0000",0.2086,"1,871.0000","6,496.0000","5,554.0000",0.7241,0
2018-01-05,2018-01-05,남원읍,"47,485.0000","1,585,700.0000",360.0000,서귀포시,seo_3,0,"19,112.0000",628.0000,"19,740.0000",0.0318,8345,2.3655,104.3947,"2,910.7413",802.4734,77.6203,"3,131.5610","2,106.1492",...,0.1535,4.1207,0.0000,9.0561,0.4452,0.0000,5.5219,49.6281,12.4555,"1,102,378.0000","1,044,956.0000","57,422.0000","3,075.0000","2,116.0000",0.2086,"1,871.0000","6,496.0000","5,554.0000",0.7127,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-27,2021-08-27,남원읍,NaN,NaN,NaN,NaN,NaN,0,"18,069.0000",619.0000,"18,688.0000",0.0331,8706,2.1500,98.8312,"3,855.6966",879.9860,73.9711,"4,619.5165","1,797.2880",...,0.5049,3.6895,0.0000,25.6886,0.1043,0.0000,12.6470,107.6707,15.8363,"986,408.0000","982,032.0000","4,376.0000","2,605.0000","1,906.0000",0.1858,"1,567.0000","6,253.0000","6,357.0000",NaN,190
2021-08-28,2021-08-28,남원읍,NaN,NaN,NaN,NaN,NaN,1,"18,069.0000",619.0000,"18,688.0000",0.0331,8706,2.1500,98.8312,"3,923.1963",531.1726,64.6442,"5,325.8746","1,856.5911",...,0.2319,4.1687,0.0000,17.7729,0.0970,0.0000,8.4076,105.9582,22.7758,"986,408.0000","982,032.0000","4,376.0000","2,605.0000","1,906.0000",0.1858,"1,567.0000","6,253.0000","6,357.0000",NaN,190
2021-08-29,2021-08-29,남원읍,NaN,NaN,NaN,NaN,NaN,1,"18,069.0000",619.0000,"18,688.0000",0.0331,8706,2.1500,98.8312,"3,954.1312",514.4904,64.5178,"5,285.8126","1,864.9638",...,0.2706,4.3124,0.0000,19.0222,0.0617,0.0000,4.9875,104.3919,25.2669,"986,408.0000","982,032.0000","4,376.0000","2,605.0000","1,906.0000",0.1858,"1,567.0000","6,253.0000","6,357.0000",NaN,190


In [30]:
df.to_csv('df_xpred.csv',index = False)

###### 나머지 변수 모두 예측 
- '배달_cnt','배달_amt','유형3_cnt','유형1_cnt','농축수산물_cnt','유형2_cnt','resd_jeju','country_3_resd_jeju'
- 'vpc_2050_daytm'

In [32]:
for x_var in tqdm(['배달_cnt','배달_amt']):
    way = dict(zip(x_model[-x_model[x_var].isna()].emd_nm, x_model[-x_model[x_var].isna()][x_var]))
    output = pd.DataFrame()
    for emd in way.keys():
        print(emd)
        var = way[emd].split("'")[1]
        tm = way[emd].split("'")[3].split('_')[1]
        mod = way[emd].split("'")[3].split('_')[0]
        res = modeling(emd,x_var,col,var,tm,mod)
        res['emd_nm'] = emd
        res = res.rename(columns = {'pred':x_var})
        output = pd.concat([output,res],axis=0)
    test = pd.merge(test, output,on = ['base_date','emd_nm'],how = 'left')

  0%|          | 0/2 [00:00<?, ?it/s]

건입동
노형동
대륜동
대정읍
대천동
동홍동
봉개동
삼도1동
삼도2동
서홍동
아라동
애월읍
연동
오라동
외도동
용담1동
용담2동
이도2동
일도2동
조천읍
중앙동
한림읍
화북동


 50%|█████     | 1/2 [12:59<12:59, 779.01s/it]

건입동
노형동
대륜동
대정읍
대천동
동홍동
삼도1동
삼도2동
삼양동
서홍동
아라동
애월읍
연동
오라동
외도동
용담1동
용담2동
이도1동
이도2동
이호동
일도1동
일도2동
정방동
한림읍
화북동


100%|██████████| 2/2 [27:24<00:00, 822.28s/it]


In [33]:
df = merge(train,test)

In [34]:
df.to_csv('df_xpred1.csv',index = False)

In [35]:
for x_var in tqdm(['유형3_cnt','유형1_cnt','농축수산물_cnt']):
    way = dict(zip(x_model[-x_model[x_var].isna()].emd_nm, x_model[-x_model[x_var].isna()][x_var]))
    output = pd.DataFrame()
    for emd in way.keys():
        print(emd)
        var = way[emd].split("'")[1]
        tm = way[emd].split("'")[3].split('_')[1]
        mod = way[emd].split("'")[3].split('_')[0]
        res = modeling(emd,x_var,col,var,tm,mod)
        res['emd_nm'] = emd
        res = res.rename(columns = {'pred':x_var})
        output = pd.concat([output,res],axis=0)
    test = pd.merge(test, output,on = ['base_date','emd_nm'],how = 'left')

  0%|          | 0/3 [00:00<?, ?it/s]

건입동
구좌읍
남원읍
노형동
대륜동
대정읍
대천동
도두동
동홍동
봉개동
삼도1동
삼도2동
삼양동
서홍동
성산읍
송산동
아라동
안덕면
애월읍
연동
영천동
예래동
오라동
외도동
용담1동
용담2동
이도1동
이도2동
이호동
일도1동
일도2동
정방동
조천읍
중문동
중앙동
천지동
표선면
한경면
한림읍
화북동
효돈동


 33%|███▎      | 1/3 [19:21<38:42, 1161.32s/it]

건입동
구좌읍
남원읍
노형동
대륜동
대정읍
대천동
도두동
동홍동
봉개동
삼도1동
삼도2동
삼양동
서홍동
성산읍
송산동
아라동
안덕면
애월읍
연동
영천동
예래동
오라동
외도동
용담1동
용담2동
이도1동
이도2동
이호동
일도1동
일도2동
정방동
조천읍
중문동
중앙동
천지동
표선면
한경면
한림읍
화북동
효돈동


 67%|██████▋   | 2/3 [37:48<19:04, 1144.95s/it]

노형동
대정읍
동홍동
삼양동
서홍동
아라동
연동
오라동
외도동
용담2동
이도2동
일도2동
조천읍
화북동


100%|██████████| 3/3 [44:10<00:00, 883.37s/it] 


In [36]:
df = merge(train,test)

In [37]:
df.to_csv('df_xpred2.csv',index = False)

In [38]:
for x_var in tqdm(['유형2_cnt','resd_jeju','country_3_resd_jeju']):
    way = dict(zip(x_model[-x_model[x_var].isna()].emd_nm, x_model[-x_model[x_var].isna()][x_var]))
    output = pd.DataFrame()
    for emd in way.keys():
        print(emd)
        var = way[emd].split("'")[1]
        tm = way[emd].split("'")[3].split('_')[1]
        mod = way[emd].split("'")[3].split('_')[0]
        res = modeling(emd,x_var,col,var,tm,mod)
        res['emd_nm'] = emd
        res = res.rename(columns = {'pred':x_var})
        output = pd.concat([output,res],axis=0)
    test = pd.merge(test, output,on = ['base_date','emd_nm'],how = 'left')

  0%|          | 0/3 [00:00<?, ?it/s]

노형동
동홍동
아라동
일도2동
화북동


 33%|███▎      | 1/3 [02:14<04:28, 134.37s/it]

구좌읍
남원읍
노형동
대륜동
대정읍
대천동
동홍동
봉개동
삼도1동
삼도2동
삼양동
성산읍
아라동
안덕면
애월읍
연동
영천동
예래동
오라동
용담1동
용담2동
이도1동
이도2동
이호동
일도2동
정방동
조천읍
중문동
표선면
한경면
한림읍
화북동


 67%|██████▋   | 2/3 [19:52<06:51, 411.51s/it]

구좌읍
남원읍
노형동
대정읍
대천동
동홍동
삼도1동
삼도2동
삼양동
아라동
안덕면
애월읍
연동
예래동
오라동
용담2동
이도2동
이호동
일도2동
정방동
조천읍
중문동
표선면
한림읍
화북동


100%|██████████| 3/3 [37:35<00:00, 751.89s/it]


In [39]:
df = merge(train,test)

In [40]:
df.to_csv('df_xpred3.csv',index = False)

In [45]:
test.to_csv('test_pred3.csv',index = False)

In [44]:
df['vpc_2050_daytm'] = df.visit_pop_jeju * df.vpc_daytm_rt

###### 'vpc_2050_daytm' 예측

In [46]:
for x_var in ['vpc_2050_daytm' ]:
    way = dict(zip(x_model[-x_model[x_var].isna()].emd_nm, x_model[-x_model[x_var].isna()][x_var]))
    output = pd.DataFrame()
    for emd in way.keys():
        print(emd)
        var = way[emd].split("'")[1]
        tm = way[emd].split("'")[3].split('_')[1]
        mod = way[emd].split("'")[3].split('_')[0]
        res = modeling(emd,x_var,col,var,tm,mod)
        res['emd_nm'] = emd
        res = res.rename(columns = {'pred':x_var})
        output = pd.concat([output,res],axis=0)
    test = pd.merge(test, output,on = ['base_date','emd_nm'],how = 'left')

건입동
구좌읍
남원읍
노형동
대륜동
대정읍
대천동
도두동
동홍동
봉개동
삼도1동
삼도2동
삼양동
서홍동
성산읍
송산동
아라동
안덕면
애월읍
연동
영천동
예래동
오라동
외도동
용담1동
용담2동
이도1동
이도2동
이호동
일도1동
일도2동
정방동
조천읍
중문동
중앙동
천지동
표선면
한경면
한림읍
화북동
효돈동


In [47]:
df = merge(train,test)

In [48]:
df.to_csv('df_xpred4.csv',index = False)
test.to_csv('test_pred4.csv',index = False)

In [50]:
df['all_cnt'] = df.유형1_cnt + df.유형2_cnt + df.유형3_cnt + df.배달_cnt + df.농축수산물_cnt

###### 'all_cnt' 예측

In [52]:
for x_var in ['all_cnt' ]:
    way = dict(zip(x_model[-x_model[x_var].isna()].emd_nm, x_model[-x_model[x_var].isna()][x_var]))
    output = pd.DataFrame()
    for emd in way.keys():
        print(emd)
        var = way[emd].split("'")[1]
        tm = way[emd].split("'")[3].split('_')[1]
        mod = way[emd].split("'")[3].split('_')[0]
        res = modeling(emd,x_var,col,var,tm,mod)
        res['emd_nm'] = emd
        res = res.rename(columns = {'pred':x_var})
        output = pd.concat([output,res],axis=0)
    test = pd.merge(test, output,on = ['base_date','emd_nm'],how = 'left')

건입동
구좌읍
남원읍
노형동
대륜동
대정읍
대천동
도두동
동홍동
봉개동
삼도1동
삼도2동
삼양동
서홍동
성산읍
송산동
아라동
안덕면
애월읍
연동
영천동
예래동
오라동
외도동
용담1동
용담2동
이도1동
이도2동
이호동
일도1동
일도2동
정방동
조천읍
중문동
중앙동
천지동
표선면
한경면
한림읍
화북동
효돈동


In [53]:
df = merge(train,test)

In [54]:
df.to_csv('df_xpred5.csv',index = False)
test.to_csv('test_pred5.csv',index = False)